<a href="https://colab.research.google.com/github/somansaleem/CLASSES-EXCERCISE-/blob/main/FA24_BBD_065.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# !pip install --upgrade visions --use-deprecated=legacy-resolver
#!pip uninstall pandas-profiling -y
!pip install visions==0.7.6 --use-deprecated=legacy-resolver

!pip install --upgrade numba --use-deprecated=legacy-resolver
# !pip install --upgrade pandas-profiling --use-deprecated=legacy-resolver
!pip install ydata-profiling --use-deprecated=legacy-resolver


Requirement already up-to-date: numba in /usr/local/lib/python3.11/dist-packages (0.61.0)


In [15]:
import pandas as pd
from pandas_profiling import ProfileReport

# Load the dataset
df = pd.read_csv("melb_data.csv")

In [16]:
# Handling Missing Values
print("Checking and Handling Missing Data")
missing_ratio = df.isnull().sum() / len(df) * 100
cols_to_remove = missing_ratio[missing_ratio > 20].index
df.drop(columns=cols_to_remove, inplace=True)
print(f"Columns have been removed due to excessive missing values: {list(cols_to_remove)}")

# Fill missing values
numeric_cols = df.select_dtypes(include=['number'])
df.fillna(numeric_cols.median(), inplace=True)

for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].fillna(df[col].mode()[0])
print("Missing values have been filled using median for numbers and mode for categorical data.")

Checking and Handling Missing Data
Columns have been removed due to excessive missing values: ['BuildingArea', 'YearBuilt']
Missing values have been filled using median for numbers and mode for categorical data.


In [17]:
# Filtering & Indexing
print("\nExtracting and Filtering Data")
if {"Suburb", "Price"}.issubset(df.columns):
    richmond_listings = df[(df["Suburb"] == "Richmond") & (df["Price"] > 1_000_000)]
    print(f"Number of Richmond properties priced above $1M: {richmond_listings.shape[0]}")
else:
    richmond_listings = pd.DataFrame()

essential_cols = {"Landsize", "Price", "Suburb", "BuildingArea"}
if essential_cols.issubset(df.columns):
    large_land_props = df.loc[df["Landsize"] > 500, ["Price", "Suburb", "BuildingArea"]]
    print(f"Total properties with land size exceeding 500 sqm: {len(large_land_props)}")
else:
    large_land_props = pd.DataFrame()
    print("Skipping land size filtering due to missing columns.")

if "Price" in df.columns:
    expensive_props = df.sort_values(by="Price", ascending=False).head(5)
    print("Top 5 of the priciest properties:")
    print(expensive_props)
else:
    expensive_props = pd.DataFrame()
    print("Unable to rank properties by price due to missing data.")


Extracting and Filtering Data
Number of Richmond properties priced above $1M: 144
Skipping land size filtering due to missing columns.
Top 5 of the priciest properties:
            Suburb         Address  Rooms Type      Price Method  \
12094     Mulgrave     35 Bevis St      3    h  9000000.0     PI   
7692    Canterbury  49 Mangarra Rd      5    h  8000000.0     VB   
9575      Hawthorn    49 Lisson Gr      4    h  7650000.0      S   
3616           Kew     15 Barry St      6    h  6500000.0      S   
12557  Middle Park     136 Page St      5    h  6400000.0      S   

            SellerG        Date  Distance  Postcode  Bedroom2  Bathroom  Car  \
12094          Hall  29/07/2017      18.8    3170.0       3.0       1.0  1.0   
7692      Sotheby's  13/05/2017       9.0    3126.0       5.0       5.0  4.0   
9575   Abercromby's  17/06/2017       5.3    3122.0       4.0       2.0  4.0   
3616         Jellis  13/08/2016       5.6    3101.0       6.0       6.0  3.0   
12557      Marshall  

In [18]:
# Data Transformation
print("\nTransforming Data for Insights")
if {"Price", "Rooms"}.issubset(df.columns):
    df["Cost_per_Room"] = df["Price"] / df["Rooms"]
    print("New column 'Cost_per_Room' created.")

if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"], errors='coerce')
    df.dropna(subset=["Date"], inplace=True)
    df["Sold_Year"] = df["Date"].dt.year
    print("Converted 'Date' column and extracted year of sale.")


Transforming Data for Insights
New column 'Cost_per_Room' created.
Converted 'Date' column and extracted year of sale.


In [19]:
# Grouping & Summarization
print("\nAggregation and Summarization")
if {"Suburb", "Price"}.issubset(df.columns):
    avg_price_per_suburb = df.groupby("Suburb")["Price"].mean()
    total_listings_per_suburb = df.groupby("Suburb")["Price"].count()
    highest_priced_suburb = avg_price_per_suburb.idxmax()
    print(f"The suburb with the highest average price: {highest_priced_suburb}")
else:
    avg_price_per_suburb = pd.Series(dtype=float)
    total_listings_per_suburb = pd.Series(dtype=int)
    highest_priced_suburb = "Data Unavailable"


Aggregation and Summarization
The suburb with the highest average price: Middle Park


In [20]:
# Data Export & Reporting
print("\nSaving Processed Data")
df.to_csv("cleaned_dataset.csv", index=False)
profile = ProfileReport(df, title="Automated Data Report", explorative=True)
profile.to_file("data_analysis_report.html")
print("Generated report and saved cleaned dataset.")



Saving Processed Data


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/ydata_profiling/model/pandas/discretize_pandas.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 9 ... 9 9 9]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  discretized_df.loc[:, column] = self._discretize_column(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Generated report and saved cleaned dataset.


In [21]:
# Advanced Insights
print("\nBonus: Additional Analysis")
if "Price" in df.columns:
    price_median = df["Price"].median()
    df["Price_Category"] = df["Price"].apply(lambda p: "Expensive" if p > price_median else "Affordable")
    price_room_correlation = df["Price"].corr(df["Rooms"]) if "Rooms" in df.columns else "N/A"
    print(f"Correlation between price and rooms: {price_room_correlation}")
else:
    price_room_correlation = "N/A"


Bonus: Additional Analysis
Correlation between price and rooms: 0.5105642446509925


In [22]:
# Save Summary
with open("summary_report.txt", "w") as f:
    f.write(f"Suburb with highest price: {highest_priced_suburb}\n")
    f.write(f"Correlation between price & rooms: {price_room_correlation}\n")
